In [3]:
# Importing libraries
import os
import numpy as np
import glob
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [5]:
# Checking for the device- i.e if torch is using CUDA or not.
# If it is not using CUDA, it will train on CPU.
# Else it will train on GPU.

# 'cuda' and 'cpu' have to be as given since they are library terms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
# Data Processing- Using transforms

transformer = transforms.Compose([
    # Resizing every image to (150, 150) for training
    transforms.Resize((150, 150)),
    
    # Random horizontally flipping images with 0.5 probability
    transforms.RandomHorizontalFlip(),
    
    # Changes [0-255] value to [0-1], also changes data type from numpy to tensor
    transforms.ToTensor(),
    
    # Changes range from [0-1] to [-1, 1]
    # Column represents RGB channel, and row represents mean and std
    # x will be replaces by the Z-score = (x-mean)/std
    transforms.Normalize([0.5, 0.5, 0.5],
                        [0.5, 0.5, 0.5])
])

In [12]:
# Data Loader

# Paths
train_path = '/home/siddhant/Pictures/Intel-image-dataset/archive/seg_train'
test_path = '/home/siddhant/Pictures/Intel-image-dataset/archive/seg_test'


train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    # Hyper parameters
    batch_size = 256,
    # Shuffle to get rid of bias
    shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    # Hyper parameters
    batch_size = 256,
    # Shuffle to get rid of bias
    shuffle = True
)

In [13]:
# Getting categories

root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [15]:
# CNN Network- In PyTorch we need to define class for a CNN- it will inherit the nn.Module class

class ConvNet(nn.Module):
    def __init__(self, num_classes = 6):
        super(ConvNet, self).__init__()
        
        # Input shape (256, 3, 150, 150) - (batch size, number of channels, height, width)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, stride = 1, padding = 1)
        # ((w-f+2p)/s)+1
        # New shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        # New shape = (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # New shape = (256, 12, 150, 150)
        
        self.pool = nn.MaxPool2d(kernel_size = 2)
        # New shape = (256, 12, 75, 75)
        
        self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3, stride = 1, padding = 1)
        # New shape = (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        # New shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        # New shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # New shape = (256, 32, 75, 75)
        
        self.fc = nn.Linear(in_features = 32*75*75, out_features = num_classes)
    
    # Feed forward function
    
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.pool(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        # Output will be [256, 32, 75, 75]
        
        output = output.view(-1, 32*75*75)
        
        output = self.fc(output)
        return output

In [16]:
# Getting model and sending it to the device
model = ConvNet(num_classes = 6).to(device)